# ROOT analyser

In [ ]:
import fitz
import cv2 as cv
import numpy as np
import json
import copy
from tqdm import tqdm

from src.viz.graphs import *
from src.viz.images import *
from src.utils import *
from src.file_reading import *
from src.detection.elements import *
from src.getFirstFrame import *

from sklearn.mixture import GaussianMixture,BayesianGaussianMixture

from IPython.display import Video

In [ ]:
large_imshow = lambda img: scaled_imshow(img,fx=0.8,fy=0.8)
simshow = lambda img: scaled_imshow(img,fx=0.3,fy=0.3)
mini_imshow = lambda img: scaled_imshow(img,fx=0.2,fy=0.2)

In [ ]:
clip = lambda i: f"clip_{i}"
clip_mp4 = lambda i: f"{clip(i)}.mp4"

In [ ]:
lower_orange = np.array([0, 100, 100])
orange = (48,91,198)
upper_orange = np.array([20, 255, 255])
lower_dark_blue = np.array([100, 50, 50])
blue = (92,38,15)
upper_dark_blue = np.array([140, 255, 255])


In [ ]:
mse = lambda a_1,a_2: np.mean((np.array(a_1)-np.array(a_2))**2)

In [ ]:
_max_color_diff = mse((0,0,0),(255,255,255))
def color_sim(a_1,a_2):
    return 1 - (mse(a_1,a_2)/_max_color_diff)

## Data

The input data is be divided into 3 groups depending on the difficulty for example:
- easy: perfect top down view, the game elements are not covered with hands when carrying them, the lighting is good
- medium: strong light at the side causing shadows,
- difficult: same as medium + a slightly angled camera, hands covering the pieces

There are 3 clips per difficulty. The data is located in a [google drive]((https://drive.google.com/drive/folders/1VrQ98TC5jPmWk1QYr3lUP3SGk_3_AEmx?usp=sharing))


In [ ]:
data_dir = ".\\data\\"
difficulties = ["easy","medium","hard"]
clip_dirs = dict([(diff,data_dir+diff+"\\") for diff in difficulties])
resized_clip_dirs = dict([(diff,data_dir+diff+f"\\resized\\") for diff in difficulties])

In [ ]:
first_frames = dict([(diff,getFirstFrame(_dir+clip_mp4(0))) for diff,_dir in clip_dirs.items()])

In [ ]:
mini_imshow(first_frames["easy"])

In [ ]:
mini_imshow(first_frames["medium"])

In [ ]:
mini_imshow(first_frames["hard"]) # TODO rotate hard clips

There are also resized clips to speed up working time.

In [ ]:
resized_first_frames = dict([(diff,getFirstFrame(_dir+clip_mp4(0))) for diff,_dir in resized_clip_dirs.items()])

In [ ]:
simshow(np.concatenate([resized_first_frames[diff] for diff in difficulties],axis=1))

The game is played between 2 factions: Eyrie Dynasties (blue birds), Marquise de Cat (orange cats). The board is a Winter Map. Because the clearings in the forest are barely differentiable, a mask was created to help with detecting static elements of the board. 

In [ ]:
game_data_dir = data_dir+"game_data\\"

In [ ]:
board_mask = cv.imread(game_data_dir+"board_mask.png")

simshow(board_mask)

- The red indicates the where the score track is. 
- The green defines where craftable items are.
- The blue shows where the clearing approximately are, with the black squares showing where building spaces are.

JSON was created to define paths on the map, done purely for drawing a graph of the map.

In [ ]:
with open("./data/game_data/board_info.json", "r") as info_file:
    board_info = json.load(info_file)

    draw_map(board_info)

To help with detection a print and play set is used with all the elements taken from [PnP PARADISE](https://www.pnpparadise.com/set1/root).

In [ ]:
board_ref = read_pdf(data_dir+'game_data\\board.pdf')

simshow(board_ref)

## Milestone 1

In this phase, the following things were detected:
- the black dice tray along with the dice on it
- the board

### Dice tray detection

In [ ]:
img = first_frames["easy"]
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

In [ ]:
mini_imshow(img)

The dice tray is all black so a simple threshold was performed.

In [ ]:
tray_cont,dice1_cont,dice2_cont,img_cont = detect_dice_tray(img)
mini_imshow(img_cont)

In [ ]:
tray = crop_contour(img,tray_cont)

dice1 = crop_contour(img,dice1_cont)
dice2 = crop_contour(img,dice2_cont)

mini_imshow(tray)
imshow(dice1)
imshow(dice2)

### Board detection

Detecting the board was harder as it has much more details.

In [ ]:
board_ref = read_pdf('.\\data\\game_data\\board.pdf')
board_gray = cv.GaussianBlur(cv.cvtColor(board_ref, cv.COLOR_BGR2GRAY),(7,7),0)

simshow(board_ref)

To achieve this steps descriptors are used, in particular the SIFT detector. To quickly match the descriptors FLANN algorithm is used.

In [ ]:
M,drawn_matches,board_cont = descriptor_detect(img,board_ref)

In [ ]:
simshow(drawn_matches)

In [ ]:
def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result

In [ ]:
crop_board = crop_contour(img,board_cont)
crop_board = rotate_image(crop_board,0.8)
simshow(crop_board)

This code doesn't have to be run much, because the board should not a lot move in the clips

In the milestone 1, there were also attempts to segment the image using a Gaussian Mixture, but they were quite slow and not effective

## Further Progress

### Tracking Game Score

Game score is tracked in the lower half of the board, by blue and orange counters.

In [ ]:
trimask = crop_contour(cv.warpPerspective(board_mask, M, (img.shape[1], img.shape[0])),board_cont)
trimask = rotate_image(trimask,1)
simshow(trimask)

In [ ]:
item_mask,clearing_mask,score_mask = trimask[:,:,1],trimask[:,:,0],trimask[:,:,2]

In [ ]:
crop_score = crop_contour(crop_board,score_mask)
simshow(crop_score)

In [ ]:
def find_cells(img,mask,thresh_arg=(55,15),hor_ker=np.ones((1,40)),ver_ker=np.ones((30,1)),hor_ver_ker=np.ones((7,7))):
    mask_cont = cv.findContours(mask,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)[0][0]
    cropped = crop_contour(img,mask_cont)
    gray = cv.cvtColor(cropped,cv.COLOR_BGR2GRAY)
    imshow(cropped)
    threshold = cv.adaptiveThreshold(
        gray,
        255,
        cv.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv.THRESH_BINARY,
        *thresh_arg
    )
    imshow(threshold)

    hor=255-cv.erode(cv.dilate(threshold,hor_ker),hor_ker)
    ver=255-cv.erode(cv.dilate(threshold,ver_ker),ver_ker)
    hor_ver = cv.morphologyEx(hor+ver,cv.MORPH_CLOSE,hor_ver_ker)
    imshow(hor_ver)
    return hor_ver,mask_cont


In [ ]:
def find_cell_contours(hor_ver):
    contours,hierarchy = cv.findContours(hor_ver,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
    i,_ = max(enumerate(contours), key=lambda i_c:cv.contourArea(i_c[1]))

    cells = []
    _,_,child,_ = hierarchy[0][i]
    j = child
    while True:
        _next,_,_,_ = hierarchy[0][j]
        cells.append(j)
        j = _next
        if _next == -1:
            break
    
    return cells,contours

    #imshow(cv.drawContours(img_contours,[largest_contour],-1,(0,0,255),2))


In [ ]:
hor_ver,score_cont = find_cells(board_ref,board_mask[:,:,2])
score_x,score_y,_,_=cv.boundingRect(score_cont)
test_score_crop = crop_contour(board_ref,score_cont)
cells,cell_cont = find_cell_contours(hor_ver)
score_contours = [cell_cont[i] for i in cells[::-1]]

In [ ]:
abs_scr_cont = [cont + [score_x,score_y] for cont in score_contours]
def warp_contours(contours,M):
    return [cv.perspectiveTransform(cont.astype(np.float64),M).astype(np.int32) for cont in contours]

In [ ]:
warp_scr_cont = warp_contours(abs_scr_cont,M)

In [ ]:
brc=np.copy(img)
simshow(cv.drawContours(brc,warp_scr_cont,-1,(255,0,0),2))

In [ ]:
def get_scores(img,warp_scr_cont):
    blue_score = None
    orange_score = None
    for cont in warp_scr_cont

In [ ]:
imshow(crop_contour(img,warp_scr_cont[0]))

In [ ]:
def calculate_color_percentage(image, lower_color, upper_color):
    # Convert the image to the HSV color space (Hue, Saturation, Value)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Create a binary mask for the specified color range
    color_mask = cv2.inRange(hsv_image, lower_color, upper_color)

    # Calculate the percentage of non-zero pixels in the mask
    total_pixels = np.prod(color_mask.shape)
    colored_pixels = np.count_nonzero(color_mask)
    percentage = (colored_pixels / total_pixels) * 100

    return percentage

In [ ]:
calculate_color_percentage(crop_contour(img,warp_scr_cont[0]),lower_dark_blue,upper_dark_blue)

In [ ]:
calculate_color_percentage(crop_contour(img,warp_scr_cont[0]),lower_orange,upper_orange)

In [ ]:
mse(orange,[  5.460956,  46.703438, 105.51114 ])

In [ ]:
def resize_contours(contours,fx=0.25,fy=0.25):
    resized = []
    for contour in contours:
        resized_contour = np.copy(contour)
        resized_contour[:, :, 0] = contour[:, :, 0] * fx
        resized_contour[:, :, 1] = contour[:, :,  1] * fy
        resized.append(resized_contour)
    return resized

In [ ]:

# for cont in warp_scr_cont:
#     cont
#     im = crop_contour(img,cont)
#     mean_c = np.mean(im, axis=(0, 1))
#     print(mean_c)
#     print(mse(mean_c,orange))
#     print(mse(mean_c,blue))
#     imshow(im)

### Tracking Card Pile

In [ ]:
card = read_pdf(game_data_dir+"card_reverse.pdf")
imshow(card)

In [ ]:
M,drawn_matches,card_cont = descriptor_detect(img,card,distance=0.5)

In [ ]:
mini_imshow(drawn_matches)

In [ ]:
crop_card = crop_contour(img,card_cont)
imshow(crop_card)

In [ ]:
ignore_dir = ".\\ignore\\"

In [ ]:
easy = cv.VideoCapture(resized_clip_dirs["easy"]+clip_mp4(0))
if easy.isOpened():
    print("Video loaded")

width,height = int(easy.get(3)), int(easy.get(4))

print(width, height)

fps = easy.get(cv2.CAP_PROP_FPS)
print(fps)

In [ ]:
contours_to_track = [card_cont,dice1_cont,dice2_cont]
static_contours = [board_cont,card_cont]

In [ ]:
start = 648
easy.set(cv.CAP_PROP_POS_FRAMES,start)
ret, frame = easy.read()
# x, y, w, h = cv.boundingRect(resized_contour)
# track_window = (x, y, w, h)
# roi = frame[y : y + h, x : x + w]

# hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
# mask = cv2.inRange(
#     hsv_roi, np.array((0.0, 60.0, 32.0)), np.array((180.0, 255.0, 255.0))
# )
# roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
# cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
# term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

In [ ]:
to_track = resize_contours(contours_to_track)
static = resize_contours(static_contours)

In [ ]:
def draw_bbox(frame, bbox, color=(255, 255, 255)):
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame, p1, p2, color, 2, 1)

In [ ]:
track = cv.VideoWriter(
    f".\\ignore\\{clip(0)}.avi",
    cv.VideoWriter_fourcc(*"DIVX"),
    fps,
    (width, height),
)

# card_tracker = cv.TrackerMIL_create()
# card_tracker.init(frame,cv.boundingRect(resized_contour))
trackers = [create_tracker("CSRT") for _ in to_track]
for tracker,contour in zip(trackers,to_track):
    tracker.init(frame,cv.boundingRect(contour))

easy.set(cv.CAP_PROP_POS_FRAMES,start)
sec = 5
i = 0
for _ in tqdm(range(int(sec*fps))):
    if not easy.isOpened():
        break
    if i > sec*fps:
        break

    ret, frame = easy.read()
    raw_frame = np.copy(frame)

    if ret:
        for cont in static:
            draw_bbox(frame, cv.boundingRect(cont), (255, 0, 0))

        for tracker in trackers:
            ok, bbox = tracker.update(raw_frame)
            if ok:
                draw_bbox(frame, bbox, (0, 255, 0))
            else:
                frame = cv2.putText(frame, 'FAILED', (0,0), cv.FONT_HERSHEY_SIMPLEX,  
                1, (0,0,255), 2, cv2.LINE_AA) 
        # hsv = cv.cvtColor(frame, cv2.COLOR_BGR2HSV)
        # dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        # ret, track_window = cv.CamShift(dst, track_window, term_crit)
        # pts = np.int0(cv.boxPoints(ret))
        track.write(frame)
        i+=1
    else:
        break

track.release()

In [ ]:
!ffmpeg -hide_banner -loglevel error -i .\ignore\clip_0.avi -y .\ignore\clip_0.mp4